In [1]:
from pathlib import Path
import scanpy as sc
import re, sys
import pandas as pd

In [2]:
def parse_sam(file):
	result = set()
	with open(file, 'r') as inputFile:
		for line in inputFile.readlines():
			fields = line.split('\t')
			acc = fields[2]
			for item in fields[19:]:
				if item.find('CB:Z:') > -1:
					barcode = re.sub(r'CB:Z:', r'', item)
				if item.find('UB:Z:') > -1:
					umi = re.sub(r'UB:Z:', '', item)
			result.add(f'{acc}\t{barcode}\t{umi}')
	return result

def generate_table(data, target_set, target_name, suffix):
	filtered_data = []
	for item in data:
		if item.split('\t')[0] in target_set:
			filtered_data.append(item)

	with open('exp_aae_denv.txt', 'w') as outputFile:
		for item in filtered_data:
			outputFile.write(item)
	table = pd.read_table('exp_aae_denv.txt', header=None, delimiter='\t')
	table.columns = ['Accession', 'Barcode', 'UMI']
	table['Accession'] = table['Accession'].map(lambda x: target_name[x])
	table = table[table['Barcode'] != '-']
	table = table[table['UMI'] != '-']
	table = table.groupby(['Accession', 'Barcode'])['UMI'].count().reset_index()
	table['Barcode'] = table['Barcode'] + suffix
	Path('exp_aae_denv.txt').unlink()
	return table

def write2csv(adata, table, target_name, suffix):
	umap_df = pd.DataFrame(
	adata.obsm['X_umap'], 
	index=adata.obs_names,
	columns=['UMAP_1', 'UMAP_2']  # 列名可自定义
	)
	umap_df = pd.concat([adata.obs['cluster'], umap_df], axis=1).reset_index()
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2']
	umap_df.to_csv('temp.csv', header=True, index=False)
	umap_df = pd.read_csv('temp.csv', header=0)
	for name in target_name.values():
		temp_column = table[table['Accession'] == name][['Barcode', 'UMI']]
		umap_df = umap_df.merge(temp_column, how='left', on='Barcode').fillna(0)
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2'] + list(target_name.values())
	umap_df.to_csv(f'exp_{suffix}.csv', index=False)
	Path('temp.csv').unlink()
	print(umap_df)

In [3]:
data = parse_sam('exp_cxtri.sam')

In [4]:
target_set = ['MW174761.1', 'ON949933.1']
target_name = {
	'MW174761.1': 'Totichi',
	'ON949933.1': 'HKIFV'
}
table = generate_table(data, target_set, target_name, '-cxtri')

In [5]:
adata = sc.read('../../../original_h5ad/cxtri_cxtri_bl_harmony.h5ad')
adata = adata[adata.obs['batch'] == 'cxtri']
write2csv(adata, table, target_name, 'cxtri')

                                  Barcode    Cluster     UMAP_1    UMAP_2  \
0     GAACGCTAT_ACACCAACG_AACAAGTGG-cxtri  EC-like-2   9.708562  0.968746   
1     GTCTTGCGT_ACCTCGACT_AACAAGTGG-cxtri         EC   5.454539 -0.403891   
2     CGGTGATTG_ACCTGCTAC_AACAAGTGG-cxtri     ISC/EB   4.224258  8.457238   
3     GCAGATGTG_ACCTGCTAC_AACAAGTGG-cxtri         EC   6.840712  1.416017   
4     TACCGTCTG_ACGTTACGA_AACAAGTGG-cxtri  EC-like-1   7.887145  2.630452   
...                                   ...        ...        ...       ...   
5543  CGCAACTAC_TGGTCAGTT_TTGTGTACG-cxtri   Cardia-2  11.776987  4.345391   
5544  GCCTACGAT_TTGAGACAG_TTGTGTACG-cxtri     ISC/EB   6.113331  8.773983   
5545  CGAGATAGT_TTGCCGTCA_TTGTGTACG-cxtri       EE-3  13.670516  6.870691   
5546  CAGAGTACA_TTGGTGACC_TTGTGTACG-cxtri  EC-like-2  11.719982  1.576641   
5547  TCTTGCTTG_TTGGTGACC_TTGTGTACG-cxtri   Cardia-2  13.965865  4.098293   

      Totichi  HKIFV  
0         0.0   13.0  
1         2.0    9.0  
2     